In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
main_path = 'drive/My Drive/Colab Notebooks/Kuzushiji/Kuzushiji-49'

In [0]:
import os
import numpy as np

In [4]:
import keras
from keras.utils import np_utils
from keras.models import Model, load_model
from keras.layers import Average

Using TensorFlow backend.


In [0]:
n_class = 49

In [0]:
test_X_file = np.load(os.path.join(main_path, 'k49-test-imgs.npz'))
test_Y_file = np.load(os.path.join(main_path, 'k49-test-labels.npz'))

In [0]:
test_X = np.expand_dims(test_X_file['arr_0'], axis = 3) / 255.0
test_Y = np_utils.to_categorical(test_Y_file['arr_0'], n_class)

In [0]:
base_model_names = ('CNN_model_1', 'CNN_model_1_100.h5')
DenseNet_names = ('DenseNet_1', 'DenseNet_1_100.h5')
save_path = os.path.join(main_path, 'model')

In [0]:
sub_models = []

In [10]:
for CNN_name, model_name in [base_model_names, DenseNet_names]:
  sub_model = load_model(os.path.join(save_path, CNN_name, model_name))
  sub_models.append(sub_model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
for i, sub_model in enumerate(sub_models):
  for index, layer in enumerate(sub_model.layers):
    sub_model.layers[index].name = layer.name + '_{}'.format(i)

In [0]:
sub_model_inputs = [
  sub_model.input
    for sub_model in sub_models
]

In [0]:
sub_model_outputs = [
  sub_model.output
    for sub_model in sub_models
]

In [0]:
outputs = Average()(sub_model_outputs)

In [0]:
model = Model(sub_model_inputs, outputs)

In [0]:
optimizer = keras.optimizers.Adam()

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_1 (InputLayer)          (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
batch_normalization_1_1 (BatchN (None, 28, 28, 1)    4           input_1_1[0][0]                  
__________________________________________________________________________________________________
activation_1_1 (Activation)     (None, 28, 28, 1)    0           batch_normalization_1_1[0][0]    
__________________________________________________________________________________________________
conv2d_1_1 (Conv2D)             (None, 28, 28, 64)   128         activation_1_1[0][0]             
__________________________________________________________________________________________________
batch_norm

In [0]:
loss, acc = model.evaluate([test_X, test_X], test_Y)

38547/38547 [==============================] - 15s 382us/step


In [0]:
print('loss: {:f}, acc.: {:f}'.format(loss, acc))

loss: 0.231771, acc.: 0.960983
